<img src="lbnl_logo.jpg">

----




# Genomics Challenge Lab - Day 2



---

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

<img src="algae.png" align="left">

# Genomics Challenge Lab - Day 2

Load yesterday's cleaned table into `rna_data`. Recall that we saved yesterday's work into a file called "rna_data_cleaned.csv". Double check that `rna_data` is what it looked like yesterday.

In [ ]:
# EXERCISE

rna_data = ...
rna_data.head()

## Normalizing the Data

### 1. What is normalization?

Previously, we noted that the values in a single column ranged from 0 to the hundreds of thousands. Since it is hard to look at numbers in this wide of a range, we want to "normalize" the data. Normalizing data means that we adjust the scale of our data by comparing our numbers on a relative scale instead of directly comparing them. This helps with data that takes on a large range of numbers, and it eliminates the issue of comparing categories that are on different scales.

For example, let's say we have two classes, Class 1 and Class 2, with class sizes of 10 and 1,000 respectively. If 10 students from each class earn a perfect score on a test, we can tell that there is a significant difference between Class 1 and Class 2's performances. However, both classes had the same number of perfect test scores when we look at the raw numbers. In order to better portray this, we want to work with relative numbers.

Let's see how this would look with the data table `classes`.

In [ ]:
# EXAMPLE

classes = pd.DataFrame([["A", 10, 10], ["B", 0, 900], ["C", 0, 90]], 
                       columns=["Grade", "Class 1", "Class 2"])
classes = classes.set_index("Grade")
classes

If we were to only look at the data associated with "A" grades, we would not know that Class 1 is performing better than Class 2. Instead, we see that both classes had the same number of scores.

In [ ]:
# EXAMPLE

classes.loc["A"]

In order to normalize the data, we could divide our `Class 1` data by the total number of students in `Class 1` and our `Class 2` data by the total number of students in `Class 2`. This tells us what _proportion_ of students received a perfect score instead of what _number_ of students did.

In [ ]:
# EXAMPLE

num_students_1 = sum(classes.iloc[:, 0])
num_students_2 = sum(classes.iloc[:, 1])

classes["Class 1"] = classes["Class 1"]/num_students_1
classes["Class 2"] = classes["Class 2"]/num_students_2

print("# of students in Class 1: " + str(num_students_1))
print("# of students in Class 2: " + str(num_students_2))

classes

Now, if we only look at the data associated with "A" grades, we can see the difference between Class 1 scores and Class 2 scores.

In [ ]:
# EXAMPLE

classes.loc["A"]

### 2. Normalization By Read Depth 

Let's apply this logic to our genomics data. Here, our columns are experiment conditions, and our rows are specific genes. It is possible that a condition has the fewest counts for all genes because the sample actually "turns on" fewer genes than other conditions, but it is also possible that the condition had fewer counts overall. In order to analyze this, we need to find out how many counts were obtained across all genes under a specific condition.

Let's define what we will be working with.
- reads: the # of genes that "turned on" (ie. the values in our data)
- read depth: the total # of reads across _all_ genes under a specific condition

**Consider this oversimplification of our `rna_data` table.** For fictional gene Cz1, we see that high light conditions produce more reads than medium light conditions. However, when compared to other genes, we notice that the relative reads are larger under the `ML` condition than under the `HL` condition.

In [ ]:
# EXAMPLE

simple_rna = pd.DataFrame([["Cz1", 50, 20], ["Cz2", 100, 1], ["Cz3", 100, 1]], 
                       columns=["Gene", "HL", "ML"])
simple_rna = simple_rna.set_index("Gene")
simple_rna

Let's write some code that calculates the read depths of all experiment conditions/samples.

- Create an empty list of read depths called `read_depths`.
- For each experiment condition (ie. each sample):
    - Calculate the read depth and call it `one_read_depth`.
    - Store it in our list, `read_depths`.
    
This is synonymous to calculating all of our class sizes.

In [ ]:
# EXERCISE

read_depths = []
num_samples = ...

for i in range(num_samples): 
    one_read_depth = ...
    read_depths.append(...)

print("These are the read depths of all samples:")
print(read_depths)

Now make a plot that shows the range of read depths across all samples. We want the x-axis to be the different samples (ie. the column labels) and the y-axis to be the read depths corresponding to those samples.

In [ ]:
# EXERCISE

x_axis = ...
y_axis = ...

plt.scatter(x_axis, y_axis)
plt.title("Samples vs. Read Depth")
plt.xlabel("Samples")
plt.ylabel("Read Depth")
plt.xticks(rotation=90)
plt.show;

Through this visualization, you can see that the read depth differs by a large amount under different conditions and samples. This is like how our class sizes differed, so it is a good thing that we have decided to normalize our data!

Now let's create a function that normalizes the data in our `rna_data` table. Write some code that calculates relative reads based on read depths. For reference, we have provided the normalization of `simple_rna` by read depth.

In [ ]:
# EXAMPLE

simple_read_depths = [250, 22]

simple_rna_by_read_depth = simple_rna / simple_read_depths
simple_rna_by_read_depth

In [ ]:
# EXERCISE

def normalizeByReadDepth(data, read_depth_list):
    return ...

Now that we have a function that normalizes by read depth (ie. by the total # of gene counts), we can normalize `rna_data` by read depth. Recall that we calculated the read depths of all experiment conditions/samples and stored it in a list called `read_depths`. Now, use `rna_data` and `read_depths` with our new function to get a normalized data table.

In [ ]:
# EXERCISE

rna_by_read_depth = normalizeByReadDepth(..., ...)
rna_by_read_depth.head()

Because each value is a proportion, the values under each column should each sum up to 1. Let's verify that.

In [ ]:
column_totals = []

for i in range(num_samples): 
    one_column_total = np.round(sum(rna_by_read_depth.iloc[:, i])) 
    column_totals.append(one_column_total)

print("These are sums of all columns:")
print(column_totals)

Let's visualize our column totals once again. Previously, our column totals were our read depths, because they were the sum of all reads under a certain condition/sample. Now, our data is in relative reads (ie. proportions), so our column totals should all be lined up at 1.

In [ ]:
x_axis = rna_data.columns
y_axis = column_totals

plt.scatter(x_axis, y_axis)
plt.title("Samples vs. Column Total")
plt.xlabel("Samples")
plt.ylabel("Column Total")
plt.xticks(rotation=90)
plt.show;

Now, our various conditions/samples are all on the same "scale", so it will be easier to compare between different light conditions and samples.

To verify this, let's compare the data for gene `Cz01g00040` before and after normalizating by read depth.

### 3. What effect did normalization have?

Create a plot that shows reads for only gene `Cz01g00040` in `rna_data`. Each row corresponds to a specific gene, so you will need to access the `Cz01g00040` row in the table. Keep in mind that this visualization would be similar to plotting the raw grade counts for the `classes` data (ie. it will not show the relative differences).

In [ ]:
# EXERCISE

x_axis = ...
y_axis = ...

plt.scatter(x_axis, y_axis)
plt.title("Samples vs. CZ01g00040 Reads")
plt.xlabel("Samples")
plt.ylabel("CZ01g00040 Reads")
plt.xticks(rotation=90)
plt.show;

**Question 1** What is the scale of `Cz01g00040` reads? How much larger is the highest read compared to the lowest read? Is there a pattern or trend in the visualization?

_Your answer here_

Now let's look at the normalized reads for gene `Cz01g00040` in `rna_by_read_depth`. Once again, `Cz01g00040` is a row in our table whose data you need to access. This visualization shows the normalized reads of this gene, so it accounts for the total number of reads each condition produced.

In [ ]:
# EXERCISE

x_axis = ...
y_axis = ...

plt.scatter(x_axis, y_axis)
plt.title("Samples vs. Normalized CZ01g00040 Reads")
plt.xlabel("Samples")
plt.ylabel("Normalized CZ01g00040 Reads")
plt.xticks(rotation=90)
plt.show;

**Question 2** What is the scale of normalized `Cz01g00040` reads? How much larger is the highest read compared to the lowest read? Is there a pattern or trend in the visualization? Did it change from the previous visualization?

_Your answer here_

Recall that we have normalized the data such that the table's values represent the reads (or gene counts) relative to the total reads (or read depth) based on the corresponding experiment conditions and sample number. As an example, we saw how gene `Cz01g00040`'s data pattern became more clear after normalization.

Even though this made our data easier to analyze, we can still do more normalization to aide the interpretation of our data.

When we normalized by read depth, we normalized by column. More specifically, we took the totals of each column and represented each data value as a proportion of each total. In order to improve our analysis, we can also do normalization by row.

**Question 3** What does each row represent? What characteristics do genes have? What do you think we could do normalization by?

_Your answer here_

### 4. Transposing Data

Before we can normalize by row, we need to go over the concept of transposing data. Recall that normalizing by column consisted of dividing our columns by their total reads. Unfortunately, computers can only divide columns by arrays/lists and cannot divide rows by arrays/lists. In order to account for this, we can transpose our data -- flip our data table such that the rows are columns and the columns are rows.

Let's take a look at the `classes` data table that we worked with previously. 

In [ ]:
# EXAMPLE

classes = pd.DataFrame([["A", 10, 10], ["B", 0, 900], ["C", 0, 90]], 
                       columns=["Grade", "Class 1", "Class 2"])
classes = classes.set_index("Grade")
classes

Try using the function `np.transpose()` so that the `classes` table has the following structure.
- Columns: "Grade", "A", "B", "C"
- Rows: "Class 1", "Class 2"
- Values: the proportion of students who got that grade across all classes
    - These values should _not_ be the same as yesterday's normalized values

In [ ]:
# EXERCISE

grade_totals = [20, 900, 90]

classes_by_grade_shares = ...
classes_by_grade_shares

Notice that the following cell applies `np.transpose()` once again. Specifically, take note of how the transpose occurs after the column division. Altogether, the whole process (denoted below) normalizes our original table, `classes`, by row.

1. Transpose the table so that the rows are columns and the columns are rows.
2. Do column division by an array/list.
3. Transpose the table again so that the structure is the same as the original table.

In [ ]:
np.transpose(classes_by_grade_shares)

Now, let's save our progress, `rna_by_read_depth`, as "rna_by_read_depth.csv".

In [ ]:
# EXERCISE

...

Notebook developed by: Sharon Greenblum & Ciara Acosta <br/>
Edited by: Kseniya Usovich